# Spark Aufgaben
1. Importe laden
2. Jupyter Spark starten und Twitter-Streams von Avro lesen
3. ETL Strecke: Avro Daten einlesen und als Delta Datei wieder raus schreiben
4. Analyse-Aufgaben erledigen 
5. Verlaufsanalyse durchführen
6. **Ausschalten der Spark-App**

## Wichtige Hinweise
1. Führe alle Anweisungen in der vorgegebenen Reihenfolge aus. Die einzelnen Programmierzellen bauen aufeinander auf.
2. **Beende unbedingt am Ende die Spark-Anwendung mit dem untersten Befehl "spark.stop()" , wenn du aufhörst an den Daten zu arbeiten.**
3. Du kannst jederzeit das Notebook wieder hochfahren, wenn du Schritt 1 & 2 (Laden der Imports & Jupyter Spark und seine Konfigurationen hochfahren) ausführen.
4. Mit **"Strg" + "Enter"** führst du einzelne Zellen direkt aus.
5. In der oberen Leiste kannst du über **"Insert"** weitere Zellen hinzufügen, um weitere Test-Funktionen zu schreiben. 

## 1. Laden der Imports
Hier werden alle benötigten Libraries für dieses Lab heruntergeladen.

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql import Row
from pyspark.sql.functions import explode
from pyspark.sql.functions import lower, col
import pyspark.sql.functions as f

from delta import *


import datetime
from datetime import datetime
import json


# use 95% of the screen for jupyter cell
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

## 2. Jupyter Spark & Konfigurationen hochfahren
Hier wird die App jupyter-spark konfiguriert und hochgefahren, welche unsere weiteren Schritte ausführt.

In [ ]:
appName="jupyter-spark"

conf = SparkConf()

# CLUSTER MANAGER
################################################################################
# set Kubernetes Master as Cluster Manager(“k8s://https://” is NOT a typo, this is how Spark knows the “provider” type).
conf.setMaster("k8s://https://kubernetes.default.svc.cluster.local:443")

# CONFIGURE KUBERNETES
################################################################################
# set the namespace that will be used for running the driver and executor pods.
conf.set("spark.kubernetes.namespace","frontend")
# set the docker image from which the Worker pods are created
conf.set("spark.kubernetes.container.image", "thinkportgmbh/workshops:spark-3.3.1")
conf.set("spark.kubernetes.container.image.pullPolicy", "Always")

# set service account to be used
conf.set("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")
# authentication for service account(required to create worker pods):
conf.set("spark.kubernetes.authenticate.caCertFile", "/var/run/secrets/kubernetes.io/serviceaccount/ca.crt")
conf.set("spark.kubernetes.authenticate.oauthTokenFile", "/var/run/secrets/kubernetes.io/serviceaccount/token")


# CONFIGURE SPARK
################################################################################
conf.set("spark.sql.session.timeZone", "Europe/Berlin")
# set driver host. In this case the ingres service for the spark driver
# find name of the driver service with 'kubectl get services' or in the helm chart configuration
conf.set("spark.driver.host", "jupyter-spark-driver.frontend.svc.cluster.local")
# set the port, If this port is busy, spark-shell tries to bind to another port.
conf.set("spark.driver.port", "29413")
# add the postgres driver jars into session
conf.set("spark.jars", "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar")
#conf.set("spark.driver.extraClassPath","/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar")
conf.set("spark.executor.extraClassPath","/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar, /opt/spark/jars/spark-avro_2.12-3.3.1.jar")
#conf.set("spark.executor.extraLibrary","/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /opt/spark/jars/kafka-clients-3.3.1.jar")

# CONFIGURE S3 CONNECTOR
conf.set("spark.hadoop.fs.s3a.endpoint", "minio.minio.svc.cluster.local:9000")
conf.set("spark.hadoop.fs.s3a.access.key", "trainadm")
conf.set("spark.hadoop.fs.s3a.secret.key", "train@thinkport")
conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

# conf.set("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1")

# CONFIGURE WORKER (Customize based on workload)
################################################################################
# set number of worker pods
conf.set("spark.executor.instances", "1")
# set memory of each worker pod
conf.set("spark.executor.memory", "1G")
# set cpu of each worker pod
conf.set("spark.executor.cores", "2")
# Number of possible tasks = cores * executores

# SPARK SESSION
################################################################################
# and last, create the spark session and pass it the config object

spark = SparkSession\
    .builder\
    .config(conf=conf) \
    .config('spark.sql.session.timeZone', 'Europe/Berlin') \
    .appName(appName)\
    .getOrCreate()

# also get the spark context
sc=spark.sparkContext
# change the log level to warning, to see less output
sc.setLogLevel('WARN')

# get the configuration object to check all the configurations the session was startet with
for entry in sc.getConf().getAll():
        if entry[0] in ["spark.app.name","spark.kubernetes.namespace","spark.executor.memory","spark.executor.cores","spark.driver.host","spark.master"]:
            print(entry[0],"=",entry[1])

## 3. Einlesen und Schreiben von Daten

### 3.1 Einlesen der Daten aus unserem S3 Speicher-Bucket 
Laden der Daten aus unserem Bucket in "s3a://twitter/avro" in einen DataFrame, um auf den Daten zu arbeiten. 

In [ ]:
df=(spark
    .read.format("avro")
    # Pfad zu Bucket
    .load("s3a://twitter/avro")
    # filter retweets raus bei denen das Hashtag truncated wurde
    .filter(f.array_contains(f.col("hashtags"),"BigData")==True)
    # repartition auf 20 um optimierter mit den wenigen cpu zu arbeiten
    .repartition(20)
   ).cache()

Erste Ausgabe der Daten in Form eines DataFrames

In [ ]:
df.show()

### 3.2 Schreiben der Daten ins Delta-Format
Hier werden die Daten direkt im Delta-Format umgewandelt und in den S3-Bucket "s3a://twitter/delta" geschrieben. Dieser Schritt ist wichtig, um die Daten passend für Trino zu abzulegen.

In [ ]:
writer_delta=(df
                .write.partitionBy("language")
                .mode("overwrite")
                .format("delta")
                .option("overwriteSchema", "true")
                .option("userMetadata", "Initial Ladung")
                .save("s3a://twitter/delta")
             )

## 4. Analyse-Aufgaben


### 4.1 Tweets anschauen und den Aufbau des Dataframes
Schau dir den Datensatz einmal genau an. Welche Spalten gibt es? Welche Datentypen sind vorhanden?

In [ ]:
df.show()

### 4.2  Das Schema des Datensatzes anzeigen 
<br>
<code> df.printSchema()</code> gibt das Schema des Datensatzes aus.

In [ ]:
df.printSchema()

### 4.3 Zählen der Tweets pro Stunde
Schreibe eine Abfrage, die **die Anzahl an Tweets pro Stunde** zählt.
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df_hourly=(df
            .withColumn("hour", f.hour(f.col("created_at")))
            .groupBy("hour")
            .count()
            .withColumnRenamed("count","total")
            .sort("hour")
          )
df_hourly.show(20)</code>
</details>
</p>


In [ ]:
df_hourly=(df  
            ...
          )

df_hourly.show(20)

### 4.4 Top 10 User nach Tweet-Anzahl
Schreibe eine Abfrage, die die **Top User** nach ihrer **Anzahl an Tweets** ausgibt. Bedenke dabei, deine Ausgabe auf **10** Einträge zu limitieren.
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df_top_user=(df
                .groupBy("user_name")
                .agg(
                    f.count("user_name").alias("numberOfTweets")
                    )
                .orderBy(f.col("numberOfTweets").desc())
                .limit(10)
                .withColumnRenamed("user_name","user")
                )
df_top_user.show()</code>
</details>
</p>

In [ ]:
df_top_user=(df
                ...
                )
df_top_user.show()


### 4.5 Umgang mit  Arrays
Für die folgenden Aufgabe wird die <code>explode</code>-Funktion benötigt. Schreibe eine Abfrage die das Hashtag-array mit <code>explode</code> teilt. Gebe dabei die Spalten "user_name", "tweet_id"und die explodierte"hashtags"- Spalte mit einem Limit von 20 Zeilen aus. 
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df_hash=(df
         .withColumn("hashtags",explode("hashtags"))
        .limit(20)
        .select("user_name", "tweet_id", "hashtags")
        )
df_hash.show()</code>
</details>
</p>

In [ ]:
df_hash=(df
         ...
        )
df_hash.show()

### 4.6 Top 5 Hashtags der Top 10 User
Schreibe eine Abfrage, die die **Top 5 der Hashtags** der **10 User** mit den **meisten Tweets** ausgibt.
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df_top5_per_user=(df_top_user
            # filter via join
            .join(df,[df_top_user.user==df.user_name],how="left")
            # hashtags array in Zeilen Einträge exploden
            .withColumn("hashtags",explode("hashtags"))
            # hashtags lowercase schreiben um Doppelungen zu entfernen
            .withColumn("hashtags", lower(col('hashtags')))
            # groupieren und counten by hashtag
            .groupBy("hashtags").agg(f.count("hashtags"))
            # rückwärts sortieren
            .sort(f.col("count(hashtags)").desc())
            # top 5 selectieren
            .limit(5) 
                 )
df_top5_per_user.show()</code>
</details>
</p>

In [ ]:
df_top5_per_user=(df_top_user
                ...
                 )
df_top5_per_user.show()

 ### 4.7 Top 10 Influencer (User mit #BigData-tweets mit den meisten Followern) 
 Schreibe eine Abfrage, die die **Top 10 Influencer** mit den **meisten Follower** zählt und sortiert anzeigt.
 <br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df_top_influencer=(df
                .groupBy("user_name")
                .agg(
                    f.max("user_follower_count").alias("follower")
                    )
                .orderBy(f.col("follower").desc())
                )
df_top_influencer.show(10)</code>
</details>
</p>

In [ ]:
df_top_influencer=(df
                ...
                )
df_top_influencer.show(10)

### 4.8 Top 10 Influencer und ihre Anzahl an tweets
Schreibe eine Abfrage, die die **Top 10 Influencer**, ihre Follower und die **Anzahl ihrer Tweets** ausgibt. außeredem soll es sortiert nach den Anzahl ihrer Follower sein. 
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df_withRetweets=(df_top_user
            # filter via join auf die Top 10 Influencer
            .join(df_top_influencer, [df_top_influencer.user_name==df_top_user.user],how="left")
            .orderBy(f.col("follower").desc())
            .limit(10)
            .drop("user_name")
            .select("user","follower","numberOfTweets")
    )
df_withRetweets.show()</code>
</details>
</p>

In [ ]:
df_withRetweets=(df_top_user
            ....
    )

df_withRetweets.show()

### Bonusaufgabe: Filter nach den Top 10 Locations und ihrem Top Hashtag
Schreibe eine Abfrage, die die **Top 10 häufigsten Locations** ausgibt und das am **zweitmeisten verwendete Hashtag** dort. Da alle unsere Daten das Hashtag #BigData beinhalten. 
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df3=(df
    .select("user_location")
    .where(~f.col("user_location").isin("","null","REMOTE","Earth"))
    .groupBy("user_location")
    .count()
    .withColumnRenamed("count","location_total")
    .orderBy(f.col("location_total").desc())
    .limit(10)
    )</code>
    
<code>df4=(df
    .select("user_location","hashtags")
    .withColumn("singletag",f.explode(f.col("hashtags")))
    .groupBy("user_location","singletag")
    .count()
    .withColumnRenamed("count","tags_total")
        )</code>
    
<code>df5=(df3.alias("a")
    .join(f.broadcast(df4.alias("b")),[df3.user_location==df4.user_location],how="left")
    .select("a.user_location","a.location_total","b.singletag","b.tags_total")      
    .withColumn("rank",f.row_number().over(Window.partitionBy("a.user_location")
    .orderBy(f.col("b.tags_total").desc())))
    .filter(f.col("rank")==1)
    .sort(f.col("location_total").desc())
    .limit(10)
    )
df5.show()</code>
</details>
</p>

## 5. Delta History and Time Travel
Führe den folgenden Code aus um die aktuelle Delta-Daten-Version upzudaten. Wenn du mehrere Versionen sehen willst schreibe öfter raus mit <code>writer_delta()</code> mit einigen Minuten Abstand.

In [ ]:
writer_delta=(df
                .write.partitionBy("language")
                .mode("overwrite")
                .format("delta")
                .option("overwriteSchema", "true")
                .option("userMetadata", "Update Ladung")
                .save("s3a://twitter/delta")
             )

### 5.1 Delta Tabelle ausgeben
Lade die Delta-Tabelle und lasse dir die ersten 2 Einträge ausgeben.
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code># Load Delta file in s3 into Delta Table Object
dt = DeltaTable.forPath(spark, "s3a://twitter/delta")
dt.toDF().show(2)</code>
</details>
</p>

In [ ]:
# Load Delta file in s3 into Delta Table Object
dt = DeltaTable.forPath(spark, "s3a://twitter/delta")
dt.toDF().show(2)


### 5.2  Erzeugen einer Historie
1. Führe mehrmals Write to Delta aus und prüfe, wie die Historie neue Einträge hinzufügt  

In [ ]:
# get the metadata for the full history of the table
fullHistoryDF = dt.history()    

# get the metadata for the last operation
lastOperationDF = dt.history(1) 

fullHistoryDF.select("version","readVersion","timestamp","userId","operation","operationParameters","operationMetrics").show()

### 5.3 Laden der aktuellen  Versionen 
Lade eine der Versionen und lasse dir alle `languages` anzeigen (via distinct().show())
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df = spark.read.format("delta").load("s3a://solution/twitter_delta")
df.select("language").distinct().show()</code>
</details>
</p>

In [ ]:
# load latest delta version


### 5.4. Laden einer ältere Versionen 
Lade eine ältere Version und bestätige, dass noch alle Daten vorhanden sind.
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df_timetravel_old = spark.read.format("delta").option("versionAsOf", 2).load("s3a://twitter/delta")
df_timetravel_old.select("language").distinct().show()
df_timetravel_old.show()</code>
</details>
</p>

In [ ]:
#load specific historic version
df_timetravel_old = spark.read.format("delta").option("versionAsOf", 2).load("s3a://twitter/delta")
df_timetravel_old.select("language").distinct().show()
df_timetravel_old.show()

### 5.5 Überschreiben von neueren Version
Überschreibe nun mit der älteren Version die Aktuellste. 
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>df_pasttopresent = (spark
                   .read.format("delta").option("versionAsOf", 0).load("s3a://twitter/delta")
                   .write.partitionBy("language").mode("overwrite").format("delta").save("s3a://twitter/delta")
                   )
df = spark.read.format("delta").load("s3a://solution/twitter_delta")
df.show()</code>
</details>
</p>

In [ ]:
# write old version back as latest


### 5.6 Zurück in die Zukunft
Kehre zurück zum aktuellsten Timestamp, indem `timestampAsOf`anstelle von `versionAsOf`verwenden und einem aktuellen timestamp, anstelle der Versionsnummer.
<br>
<br>
<details>
<summary> &#8964 Lösung </summary>
<p>
<code>f_b2future = (spark
                .read.format("delta").option("timestampAsOf", "\<aktuellsten Stand\>").load("s3a://twitter/delta")
               )
f_b2future.show()</code>
</details>
</p>

In [ ]:
f_b2future = (spark
                .read.format("delta").option("timestampAsOf", "2022-12-08 16:37:54").load("s3a://twitter/delta")
               )
f_b2future.show()

# 6. Ausschalten der Spark-App
**Bitte schließe am Ende die Spark-App wieder mit dem folgenden Befehl `spark.stop()`, wenn du fertig mit der Bearbeitung der Aufgaben bist.** 

In [ ]:
spark.stop()
